In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import jv, jn_zeros
from scipy.integrate import solve_ivp, trapezoid
import mpmath as mp
import matplotlib.animation as animation
#%%

In [2]:
N = 17
Rd = 4.282
nu = 0.0089
r = np.linspace(0, Rd, 200)
eps = 0.1
k1 = 16.4706 / Rd
ss = 0.004
a0 = 0.02599
b = 0.8
j1 = jn_zeros(1, N)
t_hat = np.arange(0.001, 0.399, 0.001)
gg = 981
gamma = 72
t0n = t_hat * np.sqrt(gg / Rd)
dt = 0.001 * np.sqrt(gg / Rd)
rr = 4
rri = 0.25
k = j1 / Rd
omega = np.sqrt(gg * k + gamma * k**3)
x0 = -0.0512 * a0 * np.exp(-0.16 * k**2) * k**2
yy = np.zeros(N)
eta = np.zeros(len(r))
path = "eps_0.1_nu_0.0089"
non_t = np.sqrt(gg / Rd)

In [3]:
def gen_data(data, temp, i=0):
    for l in range(len(r)):
        yy = k * temp[i, :] * jv(0, k * r[l])
        data[l, 1] = (trapezoid(yy, k))
    # quadgl(eta_fun, [0, inf], method='gauss-legendre')
    data[l + 1 :, 1] = (data[: l + 1, 1])
    data[:l+1,1] = np.flip(data[:l+1,1])
    return data

In [5]:
# mp.dps = 15; mp.pretty = True
# z = (
#     lambda s, i: x0[i]
#     / s
#     * (
#         1
#         - omega[i] ** 2
#         / (
#             (s + 2 * nu * k[i] ** 2) ** 2
#             - 4 * nu**1.5 * k[i] ** 3 * (s + nu * k[i] ** 2) ** 0.5
#             + omega[i] ** 2
#         )
#     )
# )

z2 = (
    lambda s, i: x0[i]
    * (
        s
        + 4 * nu * k[i] ** 2
        - (4 * k[i] ** 3 * nu / (k[i] + (k[i] ** 2 + s / nu) ** 0.5))
    )
    / (
        s
        * (
            s
            + 4 * nu * k[i] ** 2
            - (4 * k[i] ** 3 * nu / (k[i] + (k[i] ** 2 + s / nu) ** 0.5))
        )
        + omega[i] ** 2
    )
)

# temp = np.zeros((len(t_hat), len(k)), dtype=float)
temp2 = np.zeros((len(t_hat), len(k)), dtype=float)
for j in range(len(t_hat)):  # len(t_hat)
    t = t_hat[j]
    for i in range(len(k)):
        # temp[j, i] = mp.invertlaplace(
        #     lambda s: z(s, i), t, #method="stehfest", degree=60
        # )
        temp2[j, i] = mp.invertlaplace(
            lambda s: z2(s, i),
            t,  # method="stehfest", degree=60
        )
# np.save(f"temp_{nu}_{eps}-1", temp)
# np.save(f"temp2_{nu}_{eps}-1", temp2)

In [7]:
np.save(f"temp2_{nu}_{eps}-1", temp2)

In [98]:
# temp = np.load(f"temp2_{nu}_{eps}-1.npy")
plt.rcParams.update(
    {
        "font.size": 25,
        "font.family": "times new roman",
        "mathtext.fontset": "stix",
        "lines.linewidth": 4,
        "savefig.dpi": 300,
    }
)
pt = 0

fig, ax = plt.subplots(figsize=(16, 9))
data = np.zeros((2 * len(r), 2))
data[:, 0] = np.concatenate((-r, r))
for l in range(len(r)):
    yy = k * temp2[0, :] * jv(0, k * r[l])
    data[l, 1] = trapezoid(yy, k)
    # quadgl(eta_fun, [0, inf], method='gauss-legendre')
data[l + 1 :, 1] = data[: l + 1, 1]
data1 = np.loadtxt(path + f"/data_drop/wave-{0}.dat", delimiter=" ")
# data1 = np.concatenate((data1,data1))
plot = ax.scatter(data[:, 0], data[:, 1], color="blue", s=5, label="Theory")
plot1 = ax.scatter(data1[:, 1], data1[:, 0], color="red", s=5, label="Simulation")
plot2 = ax.scatter(-data1[:, 1], data1[:, 0], color="red", s=5)
ax.set_xlim([-5, 5])
ax.set_ylim([-1, 1])
plt.legend()
xtx = ax.set_title(f"Time: {t_hat[0]:<20.5g} ", loc="left")
title = ax.text(
    0.5,
    0.90,
    "",
    bbox={"facecolor": "w", "alpha": 0.5, "pad": 5},
    transform=ax.transAxes,
    ha="center",
)


def update(i):
    xtx.set_text(f"Time: {t_hat[i]:g} ")
    for l in range(len(r)):
        yy = k * temp2[i, :] * jv(0, k * r[l])
        data[l, 1] = trapezoid(yy, k)
        # quadgl(eta_fun, [0, inf], method='gauss-legendre')
    data[l + 1 :, 1] = data[: l + 1, 1]
    plot.set_offsets(data)
    data1 = np.loadtxt(path + f"/data_drop/wave-{i}.dat", delimiter=" ")
    data1[:, [0, 1]] = data1[:, [1, 0]]
    plot1.set_offsets(data1)
    data1[:, 0] *= -1
    plot2.set_offsets(data1)
    return (
        xtx,
        plot,
        plot1,
        plot2,
    )


ani = animation.FuncAnimation(
    fig, update, frames=range(1, len(t_hat)), interval=10, blit=False, repeat=True
)
writervideo = animation.FFMpegWriter(fps=10)
# ani.save(path+'/video.mp4', writer=writervideo)
plt.show()

## For 2 methods $z$ and $z_2$

In [7]:
temp = np.load(f"temp_{nu}_{eps}-1.npy")
temp2 = np.load(f"temp2_{nu}_{eps}-1.npy")
plt.rcParams.update(
    {
        "font.size": 25,
        "font.family": "times new roman",
        "mathtext.fontset": "stix",
        "lines.linewidth": 4,
        "savefig.dpi": 300,
    }
)
pt = 0

fig, ax = plt.subplots(figsize=(16, 9))
data = np.zeros((2 * len(r), 2))
data[:, 0] = np.concatenate((-r, r))
data2 = data.copy()
data[:] = gen_data(data, temp)
data2[:] = gen_data(data2, temp2)
data1 = np.loadtxt(f"epa_0.1_nu_0.0089/data_drop/wave-{0}.dat", delimiter=" ")
# data1 = np.concatenate((data1,data1))
plot = ax.scatter(
    data[:, 0], data[:, 1], color="blue", marker="^", s=15, label="Theory"
)
plot3 = ax.scatter(
    data2[:, 0], data2[:, 1], color="k", marker="*", s=15, label="Theory2"
)
plot1 = ax.scatter(data1[:, 1], data1[:, 0], color="red", s=5, label="Simulation")
plot2 = ax.scatter(-data1[:, 1], data1[:, 0], color="red", s=5)
ax.set_xlim([-5, 5])
ax.set_ylim([-0.05, 0.05])
plt.legend()
xtx = ax.set_title(f"Time: {t_hat[0]:<20.5g} ", loc="left")
title = ax.text(
    0.5,
    0.90,
    "",
    bbox={"facecolor": "w", "alpha": 0.5, "pad": 5},
    transform=ax.transAxes,
    ha="center",
)


def update(i):
    xtx.set_text(f"Time: {t_hat[i]:g} ")
    # for l in range(len(r)):
    #     yy = k * temp[i,:] * jv(0, k * r[l])

    #     data[l,1] = trapezoid(yy, k)
    #     # quadgl(eta_fun, [0, inf], method='gauss-legendre')
    # data[l+1:,1] = data[:l+1,1]
    data[:] = gen_data(data, temp, i)
    data2[:] = gen_data(data2, temp2, i)
    plot.set_offsets(data)
    plot3.set_offsets(data2)
    data1 = np.loadtxt(f"epa_0.1_nu_0.0089/data_drop/wave-{i}.dat", delimiter=" ")
    data1[:, [0, 1]] = data1[:, [1, 0]]
    plot1.set_offsets(data1)
    data1[:, 0] *= -1
    plot2.set_offsets(data1)
    return (
        xtx,
        plot,
        plot1,
        plot2,
        plot3,
    )


ani = animation.FuncAnimation(
    fig, update, frames=range(1, len(t_hat)), interval=10, blit=False, repeat=True
)
writervideo = animation.FFMpegWriter(fps=20)
# ani.save('video-3.mp4', writer=writervideo)
plt.show()

## Plot single time step

In [94]:
file = 0
ig, ax = plt.subplots(figsize=(16, 9))
data = np.zeros((2 * len(r), 2))
data[:, 0] = np.concatenate((np.flip(-r), r))
data2 = data.copy()
data[:] = gen_data(data, temp2, i=file)
# data2[:] = gen_data(data2,temp2, i=file)
data1 = np.loadtxt(path + f"/data_drop/wave-{file}.dat", delimiter=" ")
# data1 = np.concatenate((data1,data1))
plot = ax.scatter(
    data[:, 0], data[:, 1], color="blue", marker="+", s=50, label="Theory"
)
# plot3 = ax.scatter(data2[:,0], data2[:,1],color="k", marker="*", s=50, label="Theory2")
plot1 = ax.scatter(data1[:, 1], data1[:, 0], color="red", s=5, label="Simulation")
plot2 = ax.scatter(-data1[:, 1], data1[:, 0], color="red", s=5)
ax.set_xlim([-5, 5])
# ax.set_ylim([-0.05,0.05])
plt.legend()
xtx = ax.set_title(f"Time: {t_hat[0]:<20.5g} ", loc="left")
title = ax.text(
    0.5,
    0.90,
    "",
    bbox={"facecolor": "w", "alpha": 0.5, "pad": 5},
    transform=ax.transAxes,
    ha="center",
)
plt.show()
# plt.close()

## Saving Plots

In [12]:
time = np.array([0.021, 0.045, 0.076, 0.119, 0.214, 0.279, 0.357, 0.387], dtype=float)
temp2 = np.load(f"temp2_{nu}_{eps}-1.npy")
plt.rcParams.update(
    {
        "font.size": 25,
        "font.family": "times new roman",
        "mathtext.fontset": "stix",
        "lines.linewidth": 4,
        "savefig.dpi": 300,
    }
)
for t in time:
    plt.close("all")
    data = np.zeros((2 * len(r), 2))
    data[:, 0] = np.concatenate((np.flip(-r), r))
    new_t = np.around(t_hat, 3)
    i = int(np.where(new_t == t)[0])
    data[:] = gen_data(data, temp2, i=i) / 4.282
    data1 = np.loadtxt(path + f"/data_drop/wave-{int(i)}.dat", delimiter=" ") / 4.282
    plt.figure(figsize=(8, 5))
    plt.plot(data[:, 0], data[:, 1], color=[0.05, 0.6, 0.73], label="Linear")
    plt.scatter(
        data1[::5, 1], data1[::5, 0], color=[0.89, 0.15, 0.21], s=5, label="Simulation"
    )
    plt.scatter(-data1[::5, 1], data1[::5, 0], color=[0.89, 0.15, 0.21], s=5)
    plt.xlim([-0.5, 0.5])
    plt.ylim([-0.02 / 4.282, 0.02 / 4.282])
    plt.ylabel(r"$\eta$", rotation=0, labelpad=10, fontsize=35)
    plt.xlabel(r"$r$", fontsize=35)
    plt.text(
        0.03,
        0.88,
        fr"$t={t*non_t:.3g}$",
        # bbox={"boxstyle": "round", "facecolor": "w",  "pad": 0.3},
        ha="left",
        transform=plt.gca().transAxes,
    )
    plt.tight_layout()
    if t <= 0.021:
        lgnd = plt.legend()
        # lgnd.legendHandles[0]._sizes = [30]
        lgnd.legendHandles[1]._sizes = [100]
    plt.savefig(path+f"/aeps_0.006_{t:g}.eps")
    # plt.show()
    # break
    plt.close()

/tmp/ipykernel_355840/1449108282.py:42: MatplotlibDeprecationWarning: The legendHandles attribute was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use legend_handles instead.
  lgnd.legendHandles[1]._sizes = [100]
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.


In [16]:
%matplotlib agg

In [14]:
plt.close("all")